In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import folium
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='silkeh', api_key='GwBAmaelgKfgkrmuL691')

In [2]:
#IMDB
#title.basics
#tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres
titleBasics_data = (sc.textFile('D:/Workspace/_DataMining/DataSets/IMDB/titleBasics.tsv'))
titleBasics_rdd = titleBasics_data.filter(lambda l: 'primaryTitle' not in l)
                   
#create schema for df
#tbFields = titleBasicsFields
tbFields = []
tbFields.append(StructField('tconst', StringType(), True))
tbFields.append(StructField('titleType', StringType(), True))
tbFields.append(StructField('primaryTitle', StringType(), True))
tbFields.append(StructField('originalTitle', StringType(), True))
tbFields.append(StructField('isAdult', BooleanType(), True))
tbFields.append(StructField('startYear', IntegerType(), True))
tbFields.append(StructField('endYear', IntegerType(), True))
tbFields.append(StructField('runtimeMinutes', IntegerType(), True))
tbFields.append(StructField('genres', StringType(), True))                           

tbSchema = StructType(tbFields)

#create df
titleBasics_df= sqlContext.createDataFrame(titleBasics_rdd.take(10), tbSchema)

print(titleBasics_df.show(2))

#rdd to df example VELO

def add_fields(line, separator):
    fields = []
    vals = line.split(separator)
    # bike id
    fields.append(int(vals[1]))
    # day
    if '.' in vals[9]:
        date = datetime.strptime(vals[9][:-4], '%Y-%m-%d %H:%M:%S')
    else:
        date = datetime.strptime(vals[9], '%Y-%m-%d %H:%M:%S')
    fields.append(date.day)
    # month
    fields.append(date.month)
    # weekend
    if date.weekday() == 5 or date.weekday() == 6:
        fields.append(1)
    else:
        fields.append(0)
    # hour
    fields.append(date.hour)
    # minute
    fields.append(date.minute)
    try:
        # station ID
        fields.append(int(vals[4]))
        # station data
        sd = station_dict.value
        e = []
        d = sd[int(vals[4])]
        e.append(str(d[0]))
        e.append(int(d[1]))
        e.append(float(d[2]))
        e.append(float(d[3]))
        fields.extend(e)
    except:
        fields.extend([-1, 'Unknown', 0, 0.0, 0.0])
    # location length
    try:
        fields.append(float(vals[18]))
    except:
        fields.append(0.0)
    # location type
    fields.append(vals[8])
    
    return fields


# convert to a dataframe
def convert_to_df(month):
    if month == '2015-01':
        separator = '\t'
    else:
        separator = ';'
    
    month_data = sc.textFile('../DEMO_Velo/' + month + '.csv.gz')
    month_rdd = (month_data
                    .filter(lambda l: 'BikeLocationID' not in l)
                    .map(lambda l: add_fields(l, separator))
                    .filter(lambda l: len(l) == 13))
    
    # apply the schema to the RDD.
    month_df = sqlContext.createDataFrame(month_rdd, schema)

    return month_df

fields = []
fields.append(StructField('bike_id', IntegerType(), True))
fields.append(StructField('day', IntegerType(), True))
fields.append(StructField('month', IntegerType(), True))
fields.append(StructField('weekend', IntegerType(), True))
fields.append(StructField('hour', IntegerType(), True))
fields.append(StructField('minute', IntegerType(), True))
fields.append(StructField('station_id', IntegerType(), True))
fields.append(StructField('station_name', StringType(), True))
fields.append(StructField('slots', IntegerType(), True))
fields.append(StructField('lat', FloatType(), True))
fields.append(StructField('lng', FloatType(), True))
fields.append(StructField('location_length', FloatType(), True))
fields.append(StructField('location_type', StringType(), True))
schema = StructType(fields)

for i in range(1, 12):
    try:
        month = '2015-' + str(i).zfill(2)
        convert_to_df(month).registerTempTable(MONTHS[i - 1] + '_table')
        sql = 'SELECT COUNT(*) AS CNT, SUM(location_length) AS SUM FROM ' + MONTHS[i - 1] + '_table WHERE location_type = \'A\''
        bike_data.append(sqlContext.sql(sql).collect())
    except Exception as e:
        print (e)
        bike_data.append(None)

TypeError: StructType can not accept object 'tt0000001\tshort\tCarmencita\tCarmencita\t0\t1894\t\\N\t1\tDocumentary,Short' in type <class 'str'>